# Summarize downloads of the Zenodo community: nfdi4bioimage

This notebook fetches all records from the Zenodo community `nfdi4bioimage`, extracts each record URL and its download count, and saves the result to a CSV file.

In [8]:
import json
import time
from typing import Any, Dict, List, Optional
import pandas as pd
import os

BASE_URL = "https://zenodo.org/api/records"
COMMUNITY = "nfdi4bioimage"

token = os.getenv('ZENODO_API_KEY')

RAW_JSON_PATH = "zenodo_nfdi4bioimage_records.json"
CSV_PATH = "zenodo_nfdi4bioimage_downloads.csv"

In [41]:
def get_json() -> Optional[Dict[str, Any]]:
    size_per_page = 100
    total_size = None
    page = 0
    import requests
    
    collection = []
    
    while True:
        page = page + 1
        response = requests.get('https://zenodo.org/api/records',
                                params={'communities': COMMUNITY,
                                        'access_token': token,
                                        'size': size_per_page, 
                                        'page': page})    
    
        data = response.json()
        total_size = data["hits"]["total"]
        
        collection.extend(data["hits"]["hits"])

        if total_size < page * size_per_page:
            break
    return collection

## Fetch all records

In [42]:
data = get_json()

In [43]:
len(data)

140

In [56]:
# Create lists to store the data
urls = []
downloads_list = []
authors_list = []
unique_downloads_list = []

for item in data:
    # Append to lists instead of printing
    urls.append(item["links"]["self"].replace("api/records/", "record/"))
    downloads_list.append(item["stats"]["downloads"])
    unique_downloads_list.append(item["stats"]["unique_downloads"])
    authors_list.append("; ".join([a["name"] for a in item["metadata"]["creators"]]))

# Create DataFrame
df = pd.DataFrame({
    'url': urls,
    'authors': authors_list,
    'downloads': downloads_list,
    'unique_downloads': unique_downloads_list
})

# Display the DataFrame
for index, item in df.iterrows():
    print(item["downloads"], item["url"], item["authors"])

19 https://zenodo.org/record/17659149 Wetzker, Cornelia; Massei, Riccardo; Schlierf, Michael
17 https://zenodo.org/record/17639350 Soltwedel, Johannes Richard
22 https://zenodo.org/record/17604591 Moore, Josh
61 https://zenodo.org/record/17579804 Moore, Josh
111 https://zenodo.org/record/17541532 Kabjesz, Lea; Gihlein, Lea; Lampert, Mara Harriet; Haase, Robert
66 https://zenodo.org/record/17472821 Wetzker, Cornelia; Fuchs, Vanessa; Wendt, Jens; Ahmadi, Mohsen; Müller, Maximilian E.; Massei, Riccardo
95 https://zenodo.org/record/17429285 Beyer, Frauke; Unger, Michaela; Gey, Ronny; Massei, Riccardo; Wetzker, Cornelia; Nicolay, Elena; Lampert, Mara Harriet
92 https://zenodo.org/record/17424911 Kunis, Susanne
58 https://zenodo.org/record/17404133 Zobel, Thomas; Fortmann-Grote, Carsten
49 https://zenodo.org/record/17372593 Fortmann-Grote, Carsten
58 https://zenodo.org/record/17296688 Zobel, Thomas
89 https://zenodo.org/record/17251110 Stervbo, Ulrik; Brilhaus, Dominik; Vandendorpe, Justine;

In [57]:
df["downloads"].sum()

np.int64(47122)